### Experiment Tracking

In [42]:
import os
import sys
import glob
import re
import importlib
import random
sys.path.append(os.path.join(os.getcwd(), '../scripts'))
import qna_memory_jn as qna_m
import constants as c
importlib.reload(qna_m)
importlib.reload(c)

<module 'constants' from '/Users/rachitjoshi/Documents/Finbot/notebooks/../scripts/constants.py'>

#### Environment

In [8]:
os.environ["OPENAI_API_KEY"] = "6cdb659e5a9d402e80c212fe8ea26483"

In [9]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://test-chatgpt-flomoney.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [95]:
os.environ["MLFLOW_TRACKING_URI"] = Path.cwd().joinpath("../mlruns").as_posix()

In [98]:
os.environ["MLFLOW_TRACKING_URI"]

'/Users/rachitjoshi/Documents/Finbot/notebooks/../mlruns'

#### Parameters

In [84]:
import mlflow
from pathlib import Path
from datetime import date
from itertools import product

In [85]:
company_name = 'chope'

In [86]:
tuning_params = {
    'answer_max_tokens': [256, 512],
    'temperature' : [0, 0.5]
}
tuning_scripts = [qna_m.__name__, 'buffer']

#### Experiments

In [87]:
experiment_name = f"{company_name}-{date.today().strftime('%Y-%m-%d')}"

In [88]:
mlflow.set_tracking_uri(Path.cwd().joinpath("../mlruns").as_uri())

In [89]:
if mlflow.get_experiment_by_name(experiment_name) is None:
    experiment_id = mlflow.create_experiment(
        name=experiment_name,
        artifact_location=Path.cwd().joinpath("../mlruns").as_uri(),
        tags={"version":'v1', "priority":'p2'},

    )
else:
    experiment = mlflow.set_experiment(experiment_name)
    experiment_id = experiment.experiment_id

In [90]:
experiment

<Experiment: artifact_location='file:///Users/rachitjoshi/Documents/Finbot/notebooks/../mlruns', creation_time=1698470436872, experiment_id='438196797336341294', last_update_time=1698470436872, lifecycle_stage='active', name='chope-2023-10-28', tags={'priority': 'p2', 'version': 'v1'}>

In [91]:
for script in tuning_scripts:
    with mlflow.start_run(
        run_name=f'script_{script}',
        experiment_id=experiment_id,
        tags={"version": "v1", "priority": "p2"},
    #     description="parent",    
    ) as parent_run :
        mlflow.log_param("parent", "yes")
        for t, a in list(
            product(
                tuning_params["temperature"],
                tuning_params["answer_max_tokens"]
            )
        ):  
            with mlflow.start_run(
                run_name=f"params:{t, a}",
                experiment_id=experiment_id,
                nested=True,
        #         description="child1"
            ) as child_run:
                run_params = {
                    "temperature": t,
                    "answer_max_tokens": a,
                }                
                mlflow.log_param("child", "yes")
                mlflow.log_params(run_params)
                run_metrics = {
                    "metric1": random.random(),
                    "metric2": random.random(),
                }                 
                mlflow.log_metrics(run_metrics)
#                 mlflow.log_artifacts(local_dir=Path.cwd().joinpath("mlruns"))
                

In [93]:
mlflow.get_tracking_uri()

'file:///Users/rachitjoshi/Documents/Finbot/notebooks/../mlruns'

In [61]:
mlflow.end_run()

In [99]:
!mlflow ui --backend-store-uri {mlflow.get_tracking_uri()}

[2023-10-28 17:17:45 +0800] [9357] [INFO] Starting gunicorn 20.1.0
[2023-10-28 17:17:45 +0800] [9357] [INFO] Listening at: http://127.0.0.1:5000 (9357)
[2023-10-28 17:17:45 +0800] [9357] [INFO] Using worker: sync
[2023-10-28 17:17:45 +0800] [9358] [INFO] Booting worker with pid: 9358
[2023-10-28 17:17:45 +0800] [9359] [INFO] Booting worker with pid: 9359
[2023-10-28 17:17:45 +0800] [9360] [INFO] Booting worker with pid: 9360
[2023-10-28 17:17:45 +0800] [9361] [INFO] Booting worker with pid: 9361
^C
[2023-10-28 17:18:03 +0800] [9357] [INFO] Handling signal: int
[2023-10-28 17:18:03 +0800] [9358] [INFO] Worker exiting (pid: 9358)
[2023-10-28 17:18:03 +0800] [9360] [INFO] Worker exiting (pid: 9360)
[2023-10-28 17:18:03 +0800] [9359] [INFO] Worker exiting (pid: 9359)
[2023-10-28 17:18:03 +0800] [9361] [INFO] Worker exiting (pid: 9361)
